和线搜索类似，都是借助泰勒展开。但是在线搜索中，我们先用近似模型求出下降方向，然后再给定步长。但是在信頼域中，我们直接在一个有界的区域求解这个近似模型，然后迭代到下一个点，这种算法是同时找到步长和方向。

### 算法框架
$$f(x^{k}+d) = f(x^k) + \nabla f(x^k)^Td + \frac{1}{2}d^T\nabla^2f(x^k + td)d$$
$$m_k(d) = f(x^k) + \nabla f(x^k)^Td + \frac{1}{2}d^TB^kd$$
其中$B^k$是海瑟矩阵或者海瑟矩阵的近似矩阵。我们这里是在一个小的范围内使用泰勒展开近似：
$$\Omega_k = \{x^k + d \quad|\quad ||d|| \leq \Delta_k\}$$
其实就是每一步的迭代都需要求解如下子问题：
$$\min_{d\in R^n}m_{k}(d),\quad s.t. \quad ||d||\leq\Delta_k \tag{6.6.2}$$           

给一个可以用来衡量近似程度的标准：
$$\rho_k = \frac{f(x^k) - f(x^k + d^k)}{m_k(0) - m_k(d^k)}\tag{6.6.3}$$
接近1了，我们可以扩大$\Delta_k$，反之则缩小。

![](1.png)

一般情况下，$\bar{\rho_1}=0.25,\bar{\rho_2}=0.75,\gamma_1=0.25,\gamma_2=2$。接下讨论关键问题：如何求解信頼域子问题？
### 信頼域子问题求解

#### 迭代法
首先我们有如下定理：
$d^{*}$是信頼域子问题
$$\min\quad m(d)=f+g^Td+\frac{1}{2}d^TBd$$
的全局最小解当且仅当$d^{*}$是可行的并且存在$\lambda\geq 0$使得
$$(B+\lambda I)d^{n}=-g,\quad \lambda(\Delta-||d^{n}||)=0,\quad(B+\lambda I)\geq 0$$


#### 截断共轭梯度法
找到问题的近似解，先回顾一下标准的共轭梯度迭代：
对于一个二次极小话问题
$$\min_{d} q(s) = g^Ts+\frac{1}{2}s^TBs$$
给定初值$s^0=0,r^{0}=g,p^{0}=-g,$共轭梯度法的迭代过程为
$$\alpha_{k+1}=\frac{||r^k||^2}{(q^k)^TBq^k},\\
s^{k+1}=s^k+\alpha_kp^k,\\
r^{k+1}=r^{k}+\alpha_{k}Bp^{k},\\
\beta_{k}=\frac{||r^{k+1}||^2}{||r^k||^2},\\
p^{k+1}=-r^{k+1}+\beta p^{k}$$
其中得到的迭代序列$\{s^k\}$最终的输出就是二次极小化问题的，它的终止条件就是看$||r^k||$是不是足够小。

截断共轭法需要考虑矩阵$B$不是正定矩阵的情况

![](2.png)

### 应用举例
#### 同样考虑逻辑回归问题
$$\min_{x}\quad \frac{1}{m}\sum_{i=1}^{m}ln(1+exp(-b_{i}a_{i}^Tx))+\lambda||x||_2^2$$

In [3]:
import numpy as np
import matplotlib.pyplot as plt
m = 5
n = 5
A = np.random.randint(0, 10, (m,n))
B = np.random.randint(0, 10, m)
Lambda = 1/(100*m)

def sum(x):
    s = 0
    for i in range(m):
        s += np.log(1+np.exp(-B[i]*A[i]@x))
    return s
f = lambda x : (1/m)*sum(x) + Lambda*x@x

def grad_sum(x):
    s = 0
    for i in range(m):
        s += (1 - 1/(1+np.exp(-B[i]*A[i]@x))) * B[i] * A[i]
    return s
grad_f = lambda x : -(1/m)*grad_sum(x) + 2 * Lambda * x

def ggrad_sum(x):
    s = 0
    for i in range(m):
        s += (1 - 1/(1+np.exp(-B[i]*A[i]@x))) * 1/(1+np.exp(-B[i]*A[i]@x)) * np.outer(A[i], A[i])
ggrad_f = lambda x : -(1/m)*ggrad_sum(x) + 2*Lambda * np.eye(m)

In [35]:
class trust_domain():
    def __init__(self, f, grad_f, ggrad_f, step):
        self.f = f
        self.grad_f = grad_f
        self.step = step
        self.ggrad_f = ggrad_f

    def _m(self, x, d, B, k):
        return self.f(x[k]) + self.grad_f(x[k])@d + (1/2) * d@B@d

    ## find s s.t. _m(d) = min{_m(d)}
    def Steihaug_CG(self, x, Delta, B, k):
        g = self.grad_f(x[k])
        # init params
        s = np.zeros((1,m))
        r = g
        p = -g
        epsilon = 0.01   ## maybe there is a better choice 
        while(np.dot(r,r) < epsilon * np.dot(g,g)):
            alpha = (np.dot(r,r))/(p@B@p)
            s += alpha * p
            if (np.sqrt(np.dot(s,s)) > Delta):
                break
            r_next = r + alpha * B@p
            beta = np.dot(r_next,r_next)/np.dot(r,r)
            p = -r_next + beta * p
            r = r_next
        return s
    def solve(self):
        Delta = np.sqrt(n)
        x = np.zeros((self.step+1, m))
        x_init = np.linspace(1, 10, m)
        x[0] = x_init
        rho1 = 0.25
        rho2 = 0.75
        gamma1 = 0.25
        gamma2 = 2
        for k in range(self.step):
            B = self.ggrad_f(x[k])
            d = self.Steihaug_CG(x, Delta, B, k)
            rho = (self.f(x[k]) - self.f(x[k]+d))/(m(x, 0, B, k) - m(x, d, B, k))
            if (rho < rho1):
                Delta = gamma1 * Delta
            elif (rho > rho2):
                Delta = gamma2 * Delta
            else:
                Delta = Delta
            x[k+1] = x[k] + d